In [1]:

%load_ext autoreload
%autoreload 2

import torch
import tensorflow as tf

from src.turboae_adapters.dataclasses import TurboAEEncoderParameters, ENCODER_PYTORCH_PARAMETER_NAMES
from src.turboae_adapters.encoders import TFTurboAEEncoderCNN
from src.codes import turboae_binary_exact_nonsys, turboae_approximated_nonsys
from src.turboae_adapters.encoder_decoders import load_tf_turboae_decoder, TurboAEHybridEncoderDecoder

from src.channelcoding.channels import NoiselessChannel

from pprint import pprint

/home/abhijeet/anaconda3/envs/turbo/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-05-13 19:34:51.257627: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-13 19:34:51.257665: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-05-13 19:34:53.240948: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-13 19:34:53.241323: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 

In [2]:
DECODER_MODEL_PATH = '../turboae_models/629284_dta_steq2_cnn2_cnn5_enctrain2_dectrainneg15_2_dec.pt'
ENCODER_MODEL_PATH = '../turboae_models/222877_dta_steq2_cnn2_cnn5_enctrain2_dectrainneg15_2_enc.pt'


In [3]:
decoder_state_dict = torch.load(DECODER_MODEL_PATH)
encoder_state_dict = torch.load(ENCODER_MODEL_PATH)

In [4]:
def load_tf_turboae_encoder(model_path, block_len=100) -> TFTurboAEEncoderCNN:
    torch_state_dict = torch.load(model_path)
    params = TurboAEEncoderParameters.from_pytorch(torch_state_dict)
    tf_model = TFTurboAEEncoderCNN(params, block_len=block_len)
    
    return tf_model

tf_encoder = load_tf_turboae_encoder(ENCODER_MODEL_PATH)

In [10]:
tf_decoder = load_tf_turboae_decoder(DECODER_MODEL_PATH, block_len=100)

In [5]:
code_spec = turboae_binary_exact_nonsys(delay=2)
interleaver = tf_encoder.interleaver
exact_encoder = code_spec.noninterleaved_code.concat(interleaver.and_then(code_spec.interleaved_code))

approximated_code_spec = turboae_approximated_nonsys(delay=2)
approximated_encoder = approximated_code_spec.noninterleaved_code.concat(interleaver.and_then(approximated_code_spec.interleaved_code))

In [7]:
test_msg = tf.cast(tf.random.uniform(shape=[1, 100, 1], minval=0, maxval=2, dtype=tf.int32), dtype=tf.float32)

exact_encoded = exact_encoder(test_msg)
approximated_encoded = approximated_encoder(test_msg)
tf_encoded = (tf_encoder(test_msg) + 1.) / 2.

tf.cast(tf.concat([tf_encoded, exact_encoded, approximated_encoded], axis=2)[0], dtype=tf.int8)

<tf.Tensor: shape=(100, 9), dtype=int8, numpy=
array([[0, 0, 0, 1, 1, 0, 1, 1, 0],
       [0, 0, 1, 1, 0, 1, 1, 0, 1],
       [0, 1, 0, 0, 1, 0, 0, 1, 0],
       [0, 0, 1, 0, 0, 1, 0, 0, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 1, 1, 0, 1, 1, 0, 1, 0],
       [0, 0, 1, 0, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 1, 0, 0, 1, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 1, 0, 0, 1, 0, 0, 1, 0],
       [1, 1, 0, 1, 1, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0],
       [1, 0, 0, 1, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 1, 0, 0, 0],
       [1, 1, 0, 1, 1, 0, 1, 1, 0],
       [1, 0, 0, 1, 0, 0, 1, 0, 0],
       [0, 1, 1, 0, 1, 1, 0, 1, 1],
       [0, 0, 1, 0, 0, 1, 0, 0, 0],
       [1, 1, 0, 1, 1, 0, 1, 1, 1],
       [0, 1, 0, 0, 1, 0, 0, 1, 0],
       [1, 1, 0, 1, 1, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 1, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 1, 0, 0, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 1, 0, 0, 1, 1, 0, 1],
       [0, 0, 1, 

In [27]:
for i in range(1):
    test_msg = tf.cast(tf.random.uniform(shape=[1, 100, 1], minval=0, maxval=2, dtype=tf.int32), dtype=tf.float32)

    exact_encoded = exact_encoder(test_msg)
    approximated_encoded = approximated_encoder(test_msg)
    tf_encoded = (tf_encoder(test_msg) + 1.) / 2.

    matches = tf.reduce_all(tf_encoded == exact_encoded, axis=2)[0]
    # pprint(matches)
    middle_does_match = tf.reduce_all(matches[2:-2])
    total_matches = tf.reduce_sum(tf.cast(matches, dtype=tf.int32))
    print(f"Middle matches {middle_does_match.numpy()}")
    print(f"# of matches {total_matches.numpy()}")
    # pprint(matches)
    # together = tf.cast(tf.concat([tf_encoded[0], exact_encoded[0]], axis=1), dtype=tf.int32)
    # pprint(together.numpy())

Middle matches True
# of matches 96
Middle matches True
# of matches 97
Middle matches True
# of matches 96
Middle matches True
# of matches 96
Middle matches True
# of matches 96
Middle matches True
# of matches 97
Middle matches True
# of matches 96
Middle matches True
# of matches 96
Middle matches True
# of matches 96
Middle matches True
# of matches 97


In [3]:
a = tf.constant(1)
isinstance(a, tf.Tensor)

True

In [ ]:
code_spec = turboae_binary_exact_nonsys(delay=2)
enc_dec = TurboAEHybridEncoderDecoder(
    code_spec.noninterleaved_code,
    code_spec.interleaved_code,
    DECODER_MODEL_PATH,
    channel=NoiselessChannel(1.),
    block_len=100
)
enc_dec2 = TurboAEHybridEncoderDecoder(
    code_spec.noninterleaved_code,
    code_spec.interleaved_code,
    DECODER_MODEL_PATH,
    channel=NoiselessChannel(1.),
    block_len=100
)

enc_dec.save_weights('../tmp/test_save_weights.h5')
enc_dec2.load_weights('../tmp/test_save_weights.h5')


test_msg = tf.cast(tf.random.uniform(shape=[20, 100, 1], minval=0, maxval=2, dtype=tf.int32), dtype=tf.float32)
tf.reduce_all(enc_dec(test_msg) == enc_dec2(test_msg))

../tmp/test_save_weights.h5


<tf.Tensor: shape=(), dtype=bool, numpy=True>